In [20]:
from bs4 import BeautifulSoup
import requests
import yaml
import time

In [19]:
LOGIN_URL = "https://login.ah.nl/secure/oauth/authorize?client_id=appie&redirect_uri=appie://login-exit&response_type=code"
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.0.0"
CAPTCHA_SOLVER_URL = "http://2captcha.com/in.php?key={api_key}&method=hcaptcha&sitekey={data_sitekey}&pageurl={url}&invisible=1&userAgent={user_agent}&json=1"
CAPTCHA_SOLVER_RESULT_URL = "http://2captcha.com/res.php?key={api_key}&action=get&id={id}&json=1"


In [12]:
with open("config.yml", "r") as f:
    config = yaml.safe_load(f)

In [21]:

response = requests.get(LOGIN_URL, headers={"User-Agent": user_agent})
html_doc = response.text
soup = BeautifulSoup(html_doc, "html.parser")


In [22]:
# get csrf token
csrf_token = soup.find("input", {"name": "_csrf"})["value"]

# get hcaptchaDiv and the data-sitekey
hcaptchaDiv = soup.find("div", {"id": "hcaptchaDiv"})
data_sitekey = hcaptchaDiv["data-sitekey"]
captcha_solver_url = CAPTCHA_SOLVER_URL.format(
    api_key=config["2captcha"]["api_key"],
    data_sitekey=data_sitekey,
    url=response.url,
    user_agent=user_agent,
)
# send get request to 2captcha
reponse_solver = requests.get(captcha_solver_url)
captcha_id = reponse_solver.json()["request"]
print(captcha_id)

# wait 20 seconds with countdown but on the same line
for i in range(20, 0, -1):
    print(i, end=" ")
    time.sleep(1)
print()

# get the result
captcha_solver_result_url = CAPTCHA_SOLVER_RESULT_URL.format(
    api_key=config["2captcha"]["api_key"], id=captcha_id
)
response_solver_result = requests.get(captcha_solver_result_url)
captcha_result = response_solver_result.json()["request"]
print(captcha_result)




71973041135
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 
CAPCHA_NOT_READY
